# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [7]:
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
import logging
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
import os
import azureml.core
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core.model import Model


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'insurance'

experiment=Experiment(ws, experiment_name)

# data that will be used from this experiment was downloaded from Zindi Africa. It was designed by Data Science Nigeria
# .. for a hackathon. In the dataset we have various features of buildings and the goal is to predict
#.. whether a building will have an insurance claim over a given period of time
#.. the link is here: "https://zindi.africa/competitions/data-science-nigeria-2019-challenge-1-insurance-prediction/data"
data=pd.read_csv("train_data.csv")
data.head(5)

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
0,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,.,1053,0
1,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4,1053,0
2,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,.,1053,0
3,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,.,1053,0
4,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3,1053,0


In [9]:
#creating compute cluster
compute_cluster_name = "ikecompute"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [13]:
train, test = train_test_split(data, test_size=0.2, random_state=1 )

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'insurance-project', overwrite = True,  show_progress = True)

Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_cb358d09437449cfbca320708bd65f70

In [14]:
#accessing the split data
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("insurance-project/train_data.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "enable_early_stopping":True,
    "max_concurrent_iterations":5,
    "primary_metric": "accuracy",
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='Claim',
    n_cross_validations=4)

In [16]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [24]:
from azureml.core.run import Run
#Getting best run
best_run, fitted_model=remote_run.get_output()
print(best_run)
print(fitted_model)
best_run.get_tags()
print(best_run.properties['model_name'])

Run(Experiment: insurance,
Id: AutoML_022248bb-c4c6-4c3f-a420-664665ca11e4_27,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_features='log2',
                                                                                                    max_leaf_nodes=None,
                           

In [ ]:
#TODO: Save the best model
import joblib
os.makdirs

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service